## 2. Analyze CORD-19 Datasets
### COVID-19 Open Research Dataset Challenge (CORD-19) Working Notebooks

This is a working notebook for the [COVID-19 Open Research Dataset Challenge (CORD-19)](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) to help you jump start your analysis of the CORD-19 dataset.  

<img src="https://miro.medium.com/max/3648/1*596Ur1UdO-fzQsaiGPrNQg.png" width="700"/>

Attributions:
* The licenses for each dataset used for this workbook can be found in the *all _ sources _ metadata csv file* which is included in the [downloaded dataset](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/download).  
* For the 2020-03-03 dataset: 
  * `comm_use_subset`: Commercial use subset (includes PMC content) -- 9000 papers, 186Mb
  * `noncomm_use_subset`: Non-commercial use subset (includes PMC content) -- 1973 papers, 36Mb
  * `biorxiov_medrxiv`: bioRxiv/medRxiv subset (pre-prints that are not peer reviewed) -- 803 papers, 13Mb
* When using Databricks or Databricks Community Edition, a copy of this dataset has been made available at `/databricks-datasets/COVID/CORD-19`
* This notebook is freely available to share, licensed under [CC BY 3.0](https://creativecommons.org/licenses/by/3.0/us/)

#### Configure Parquet Path Variables
Save the data in Parquet format at: `/tmp/dennylee/COVID/CORD-19/2020-03-13/`

In [3]:
# Configure Parquet Paths in Python
comm_use_subset_pq_path = "/tmp/dennylee/COVID/CORD-19/2020-03-13/comm_use_subset.parquet"
noncomm_use_subset_pq_path = "/tmp/dennylee/COVID/CORD-19/2020-03-13/noncomm_use_subset.parquet"
biorxiv_medrxiv_pq_path = "/tmp/dennylee/COVID/CORD-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv.parquet"
json_schema_path = "/databricks-datasets/COVID/CORD-19/2020-03-13/json_schema.txt"

# Configure Path as Shell Enviroment Variables
import os
os.environ['comm_use_subset_pq_path']=''.join(comm_use_subset_pq_path)
os.environ['noncomm_use_subset_pq_path']=''.join(noncomm_use_subset_pq_path)
os.environ['biorxiv_medrxiv_pq_path']=''.join(biorxiv_medrxiv_pq_path)
os.environ['json_schema_path']=''.join(json_schema_path)

#### Read Parquet Files
As these are correctly formed JSON files, you can use `spark.read.json` to read these files.  Note, you will need to specify the *multiline* option.

In [5]:
# Reread files
comm_use_subset = spark.read.format("parquet").load(comm_use_subset_pq_path)
noncomm_use_subset = spark.read.format("parquet").load(noncomm_use_subset_pq_path)
biorxiv_medrxiv = spark.read.format("parquet").load(biorxiv_medrxiv_pq_path)

In [6]:
# Count number of records
comm_use_subset_cnt = comm_use_subset.count()
noncomm_use_subset_cnt = noncomm_use_subset.count()
biorxiv_medrxiv_cnt = biorxiv_medrxiv.count()

# Print out
print ("comm_use_subset: %s, noncomm_use_subset: %s, biorxiv_medrxiv: %s" % (comm_use_subset_cnt, noncomm_use_subset_cnt, biorxiv_medrxiv_cnt))

comm_use_subset: 9000, noncomm_use_subset: 1973, biorxiv_medrxiv: 803

In [7]:
%sh 
cat /dbfs$json_schema_path

# JSON schema of full text documents


{
 "paper_id": <str>, # 40-character sha1 of the PDF
 "metadata": {
 "title": <str>,
 "authors": [ # list of author dicts, in order
 {
 "first": <str>,
 "middle": <list of str>,
 "last": <str>,
 "suffix": <str>,
 "affiliation": <dict>,
 "email": <str>
 },
 ...
 ],
 "abstract": [ # list of paragraphs in the abstract
 {
 "text": <str>,
 "cite_spans": [ # list of character indices of inline citations
 # e.g. citation "[7]" occurs at positions 151-154 in "text"
 # linked to bibliography entry BIBREF3
 {
 "start": 151,
 "end": 154,
 "text": "[7]",
 "ref_id": "BIBREF3"
 },
 ...
 ],
 "ref_spans": <list of dicts similar to cite_spans>, # e.g. inline reference to "Table 1"
 "section": "Abstract"
 },
 ...
 ],
 "body_text": [ # list of paragraphs in full body
 # paragraph dicts look the same as above
 {
 "text": <str>,
 "cite_spans": [],
 "ref_spans": [],
 "eq_spans": [],
 "section": "Introduction"
 },
 ...
 {
 ...,
 "section": "Conclusion"
 }
 ],
 "bib_entries": {
 "BIBREF0": {
 "ref_id": <str>,
 "title": <str>,
 "authors": <list of dict> # same structure as earlier,
 # but without `affiliation` or `email`
 "year": <int>,
 "venue": <str>,
 "volume": <str>,
 "issn": <str>,
 "pages": <str>,
 "other_ids": {
 "DOI": [
 <str>
 ]
 }
 },
 "BIBREF1": {},
 ...
 "BIBREF25": {}
 },
 "ref_entries":
 "FIGREF0": {
 "text": <str>, # figure caption text
 "type": "figure"
 },
 ...
 "TABREF13": {
 "text": <str>, # table caption text
 "type": "table"
 }
 },
 "back_matter": <list of dict> # same structure as body_text
 }
}

In [8]:
comm_use_subset.createOrReplaceTempView("comm_use_subset")
comm_use_subset.printSchema()

root
-- abstract: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- back_matter: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- bib_entries: struct (nullable = true)
 |-- BIBREF0: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMCID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF1: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMCID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- arXiv: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF10: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMCID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: st

In [9]:
%sql
select paper_id, metadata.title, metadata.authors, metadata from comm_use_subset limit 10

paper_id,title,authors,metadata
bb8a05062237c64d231f3ba922b8ec26c7f32eaa,,List(),"List(List(), )"
902ec7158906ac390bdc04cd55350a12c8a39281,The Evolution of Poxvirus Vaccines,"List(List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), , Lucas, Sánchez-Sampedro, List(), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), perdigue@cnb.csic.esb.p., Beatriz, Perdiguero, List(), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), , Ernesto, Mejí As-Pé Rez, List(), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), , Juan, Garcí A-Arriaza, List(), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), mauro.dipilato@cnb.csic.esm.d.p., Mauro, Pilato, List(Di), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), mesteban@cnb.csic.es, Mariano, Esteban, List(), ))","List(List(List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), , Lucas, Sánchez-Sampedro, List(), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), perdigue@cnb.csic.esb.p., Beatriz, Perdiguero, List(), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), , Ernesto, Mejí As-Pé Rez, List(), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), , Juan, Garcí A-Arriaza, List(), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), mauro.dipilato@cnb.csic.esm.d.p., Mauro, Pilato, List(Di), ), List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), mesteban@cnb.csic.es, Mariano, Esteban, List(), )), The Evolution of Poxvirus Vaccines)"
e5407e15d4e044411687f36176bc9d27bb6a3fd4,TRAF molecules in cell signaling and in human diseases,"List(List(List(null, null, null), , Ping, Xie, List(), ))","List(List(List(List(null, null, null), , Ping, Xie, List(), )), TRAF molecules in cell signaling and in human diseases)"
b9f063ab66715b75706b11ab3b0a2af52294cd5b,SheddomeDB: the ectodomain shedding database for membrane-bound shed markers,"List(List(List(Asia Pacific Bioinformatics Conference Shenzhen, , List(null, China, null, null, null, null)), , Wei-Sheng, Tien, List(), ), List(List(Asia Pacific Bioinformatics Conference Shenzhen, , List(null, China, null, null, null, null)), , Jun-Hong, Chen, List(), ), List(List(Asia Pacific Bioinformatics Conference Shenzhen, , List(null, China, null, null, null, null)), , Kun-Pin, Wu, List(), ))","List(List(List(List(Asia Pacific Bioinformatics Conference Shenzhen, , List(null, China, null, null, null, null)), , Wei-Sheng, Tien, List(), ), List(List(Asia Pacific Bioinformatics Conference Shenzhen, , List(null, China, null, null, null, null)), , Jun-Hong, Chen, List(), ), List(List(Asia Pacific Bioinformatics Conference Shenzhen, , List(null, China, null, null, null, null)), , Kun-Pin, Wu, List(), )), SheddomeDB: the ectodomain shedding database for membrane-bound shed markers)"
47dffc938eba5dd824d4f715af096791b0d2d04b,Surfactant Protein D in Respiratory and Non-Respiratory Diseases,"List(List(List(null, null, null), , Mehdi, Mirsaeidi, List(), ), List(List(null, null, null), , Uday, Kishore, List(), ), List(List(null, null, null), , Taruna, Madan, List(), ), List(List(null, null, null), glsorensen@health.sdu.dk, Grith, Sorensen, List(L), ))","List(List(List(List(null, null, null), , Mehdi, Mirsaeidi, List(), ), List(List(null, 

In [10]:
%sql
select paper_id, metadata.title, explode(metadata.authors) from comm_use_subset limit 10

paper_id,title,col
902ec7158906ac390bdc04cd55350a12c8a39281,The Evolution of Poxvirus Vaccines,"List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), , Lucas, Sánchez-Sampedro, List(), )"
902ec7158906ac390bdc04cd55350a12c8a39281,The Evolution of Poxvirus Vaccines,"List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), perdigue@cnb.csic.esb.p., Beatriz, Perdiguero, List(), )"
902ec7158906ac390bdc04cd55350a12c8a39281,The Evolution of Poxvirus Vaccines,"List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), , Ernesto, Mejí As-Pé Rez, List(), )"
902ec7158906ac390bdc04cd55350a12c8a39281,The Evolution of Poxvirus Vaccines,"List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), , Juan, Garcí A-Arriaza, List(), )"
902ec7158906ac390bdc04cd55350a12c8a39281,The Evolution of Poxvirus Vaccines,"List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), mauro.dipilato@cnb.csic.esm.d.p., Mauro, Pilato, List(Di), )"
902ec7158906ac390bdc04cd55350a12c8a39281,The Evolution of Poxvirus Vaccines,"List(List(Consejo Superior de Investigaciones Cientí ficas (CSIC), , List(Madrid-28049, Spain, null, null, null, null)), mesteban@cnb.csic.es, Mariano, Esteban, List(), )"
e5407e15d4e044411687f36176bc9d27bb6a3fd4,TRAF molecules in cell signaling and in human diseases,"List(List(null, null, null), , Ping, Xie, List(), )"
b9f063ab66715b75706b11ab3b0a2af52294cd5b,SheddomeDB: the ectodomain shedding database for membrane-bound shed markers,"List(List(Asia Pacific Bioinformatics Conference Shenzhen, , List(null, China, null, null, null, null)), , Wei-Sheng, Tien, List(), )"
b9f063ab66715b75706b11ab3b0a2af52294cd5b,SheddomeDB: the ectodomain shedding database for membrane-bound shed markers,"List(List(Asia Pacific Bioinformatics Conference Shenzhen, , List(null, China, null, null, null, null)), , Jun-Hong, Chen, List(), )"
b9f063ab66715b75706b11ab3b0a2af52294cd5b,SheddomeDB: the ectodomain shedding database for membrane-bound shed markers,"List(List(Asia Pacific Bioinformatics Conference Shenzhen, , List(null, China, null, null, null, null)), , Kun-Pin, Wu, List(), )"


In [11]:
%sql
select paper_id, min(country) as AuthorCountry
  from (
select paper_id, authors.affiliation.location.country as country
  from (
    select paper_id, metadata.title as title, explode(metadata.authors) as authors from comm_use_subset 
  ) a
 where authors.affiliation.location.country is not null  
) x
group by paper_id


paper_id,AuthorCountry
0a1533470817bc5ef0d0d0af56386a96b505dc0d,Norway
0ddcfc9bedfb0a87a7221dd2448bd41d3ba9cc51,United Kingdom
1638100b254164ee9af7d66be61794a7efa07b78,PR China
183e393843de9d6c653897f1039ad10af9750347,Canada
3422bab59932dcb046a0dfe931eb71173130979f,Republic of Korea
367e3d844bd06915e08d9082a3b720dcc6ac845f,Japan
39a1d7e4cf03a63037800c831965232d1d259e0f,Canada
468d8047a0283ba355246ed938fac8434a589a26,USA
4f7c1aeb6e5b1e35d6be98d478259a83a8dd6993,France
5f938778c47463eaa1e15fd63c0656cf830220e3,China


In [12]:
%sql
select authors.affiliation.location.country as country, count(distinct paper_id) as papers 
  from (
    select paper_id, metadata.title as title, explode(metadata.authors) as authors from comm_use_subset 
  ) a
group by country

country,papers
"Spain, UNITED STATES",1
United Kingdom A R,1
Utah,1
"Ghana, Kenya",1
Russia,28
"The Netherlands, The Netherlands",4
Paraguay,2
USa,1
"France., France",1
israel,2


In [13]:
%sql
select *
  from (
    select paper_id, metadata.title as title, explode(metadata.authors) as authors from comm_use_subset 
  ) a
where authors.affiliation.location.country like '%USA, USA, USA, USA%'

paper_id,title,authors
2a6a9de82dc0494f32530e1ee8ee7509367a04fd,Building International Genomics Collaboration for Global Health Security,"List(List(Blood Systems Research Institute, Los Alamos National Laboratory, List(null, USA, USA, USA, USA, null, null, NM, Metabiota, Los Alamos)), , Nathan, Wolfe, List(), )"
2a6a9de82dc0494f32530e1ee8ee7509367a04fd,Building International Genomics Collaboration for Global Health Security,"List(List(Blood Systems Research Institute, Los Alamos National Laboratory, List(null, USA, USA, USA, USA, null, null, NM, Metabiota, Los Alamos)), , Paras, Jain, List(), )"
2a6a9de82dc0494f32530e1ee8ee7509367a04fd,Building International Genomics Collaboration for Global Health Security,"List(List(Blood Systems Research Institute, Los Alamos National Laboratory, List(null, USA, USA, USA, USA, null, null, NM, Metabiota, Los Alamos)), , Eric, Delwart, List(), )"
2a6a9de82dc0494f32530e1ee8ee7509367a04fd,Building International Genomics Collaboration for Global Health Security,"List(List(Blood Systems Research Institute, Los Alamos National Laboratory, List(null, USA, USA, USA, USA, null, null, NM, Metabiota, Los Alamos)), hhcui@lanl.gov, Helen, Cui, List(H), )"
2a6a9de82dc0494f32530e1ee8ee7509367a04fd,Building International Genomics Collaboration for Global Health Security,"List(List(Blood Systems Research Institute, Los Alamos National Laboratory, List(null, USA, USA, USA, USA, null, null, NM, Metabiota, Los Alamos)), , Tracy, Erkkila, List(), )"
2a6a9de82dc0494f32530e1ee8ee7509367a04fd,Building International Genomics Collaboration for Global Health Security,"List(List(Blood Systems Research Institute, Los Alamos National Laboratory, List(null, USA, USA, USA, USA, null, null, NM, Metabiota, Los Alamos)), , Patrick, Chain, List(S G), )"
2a6a9de82dc0494f32530e1ee8ee7509367a04fd,Building International Genomics Collaboration for Global Health Security,"List(List(Blood Systems Research Institute, Los Alamos National Laboratory, List(null, USA, USA, USA, USA, null, null, NM, Metabiota, Los Alamos)), , Momchilo, Vuyisich, List(), )"


In [14]:
# papers by Author Country
papersByCountry = spark.sql("""
select paper_id, min(country) as AuthorCountry
  from (
select paper_id, authors.affiliation.location.country as country
  from (
    select paper_id, metadata.title as title, explode(metadata.authors) as authors from comm_use_subset 
  ) a
 where authors.affiliation.location.country is not null  
) x
group by paper_id
""")

# Create temp view
papersByCountry.createOrReplaceTempView("papersByCountry")

In [15]:
mapCountryCleansed = spark.read.options(header='true', inferSchema='true', sep='|').csv("tmp/dennylee/mappings/mapCountryCleansed")
mapCountryCleansed.createOrReplaceTempView("mapCountryCleansed")

In [16]:
%sql
select m.Alpha3, count(distinct p.paper_id) as papers
  from papersByCountry p
    left join mapCountryCleansed m
      on m.AuthorCountry = p.AuthorCountry
 group by m.Alpha3

Alpha3,papers
HTI,1
PSE,1
POL,25
JAM,2
BRA,115
JOR,6
CUB,2
FRA,222
COD,3
URY,1


#### Reference
Refrence code for mapping "country" values

In [20]:
mapCountry = spark.sql("""select distinct AuthorCountry from papersByCountry""")
mapCountry.createOrReplaceTempView("mapCountry")
mapCountry.count()

Out[21]: 265

In [21]:
%sql
select * from mapCountry order by AuthorCountry

AuthorCountry
12 Korea
2 Republic of Korea
Algeria
Argentina
"Argentina, China"
Australia
"Australia, Australia"
"Australia, Canada"
Austria
Bahrain


In [22]:
dbutils.fs.put("/tmp/dennylee/mappings/mapCountryCleansed", """
AuthorCountry|Alpha2|Alpha3
12 Korea|KR|KOR
2 Republic of Korea|KR|KOR
Algeria|DZ|DZA
Argentina|AR|ARG
Argentina, China|AR|ARG
Australia|AU|AUS
Australia, Australia|AU|AUS
Australia, Canada|AU|AUS
Austria|AT|AUT
Bahrain|BH|BHR
Bangladesh|BD|BGD
Belgium|BE|BEL
Belgium;, France|BE|BEL
Benin|BJ|BEN
Botswana|BW|BWA
Brasil|BR|BRA
Brazil|BR|BRA
Brazil (, Brazil|BR|BRA
Brazil Correspondence|BR|BRA
Brazil., Brazil|BR|BRA
Bulgaria|BGR
CYPRUS|CY|CYP
California|US|USA
Cambodia|KH|KHM
Cambodia, France|KH|KHM
Cameroon|CM|CMR
Cameroun|CM|CMR
Canada|CA|CAN
Canada, France|CA|CAN
Canada, United States of America|CA|CAN
Canada;|CA|CAN
Centre, France|FR|FRA
Chile|CL|CHL
China|CN|CHN
China SAR|HK|HKG
China, 6 Ministry|CN|CHN
China, China|CN|CHN
China, People's Republic of China|CN|CHN
China-Japan|CN|CHN
China-Japan, China|CN|CHN
China., China|CN|CHN
Colombia|CO|COL
Croatia|HR|HRV
Croatia, Croatia|HR|HRV
Cuba|CU|CUB
Cyprus|CY|CYP
Czech Republic|CZ|CZE
Democratic Republic of Congo|CD|COD
Democratic Republic of the Congo|CD|COD
Denmark|DK|DNK
Denmark, Sweden|DK|DNK
Denmark;, The Netherlands|DK|DNK
Ecuador|EC|ECU
Egypt|EG|EGY
Egypt, Republic of Congo|EG|EGY
England, UK|GB|GBR
England, United Kingdom|GB|GBR
España. Correspondence|ES|ESP
Estonia|EE|EST
Ethiopia|ET|ETH
FRANCE|FR|FRA
Finland|FI|FIN
France|FR|FRA
France Correspondence|FR|FRA
France;, France|FR|FRA
Gabon|GA|GAB
Gdansk Poland|PL|POL
Georgia|GE|GEO
Georgia, USA|US|USA
Germany|DE|DEU
Germany, Canada|DE|DEU
Germany, Germany|DE|DEU
Germany, Germany, Germany|DE|DEU
Germany, SPAIN|DE|DEU
Germany, Sweden|DE|DEU
Germany;, USA., USA|DE|DEU
Ghana|GH|GHA
Greece|GR|GRC
Grenada|GD|GRD
Guatemala|GT|GTM
Guinea|GN|GIN
Haiti|HT|HTI
Hungary|HU|HUN
India|IN|IND
India, Pakistan;, Pakistan|IN|IND
India. *Correspondence|IN|IND
India;, Norway|IN|IND
Indonesia|ID|IDN
Iran|IR|IRN
Iran, Malaysia|IR|IRN
Iraq|IQ|IRQ
Ireland|IE|IRL
Israel|IL|ISR
Israel, USA|IL|ISR
Italy|IT|ITA
Italy, United States, Germany, United States|IT|ITA
Jamaica|JM|JAM
Jamaica (|JM|JAM
Japan|JP|JPN
Japan Racing Association, Japan|JP|JPN
Japan, Japan|JP|JPN
Jordan|JO|JOR
Jordan, Jordan|JO|JOR
Kazakhstan|KZ|KAZ
Kelantan|MY|MYS
Kenya|KE|KEN
Kenya, Kenya|KE|KEN
Kingdom of Bahrain|BH|BHR
Korea|KR|KOR
Korea Correspondence, UK|KR|KOR
Korea, Korea, South Korea|KR|KOR
Kuwait|KW|KWT
Kyrgyzstan|KG|KGZ
Lebanon|LB|LBN
Liberia|LR|LBR
Lin-, Taiwan|TW|TWN
Lithuania|LT|LTU
Luxembourg|LU|LUX
Madagascar|MG|MDG
Malawi|MW|MWI
Malaysia|MY|MYS
Mali|ML|MLI
Mexico|MX|MEX
Mongolia|MN|MNG
Morocco|MA|MAR
México|MX|MEX
Nepal|NP|NPL
Nepal;|NP|NPL
Netherlands|NL|NLD
New Jersey|US|USA
New Zealand|NZ|NZL
Nicaragua|NI|NIC
Niger|NE|NER
Nigeria|NG|NGA
Norway|NO|NOR
Oman|OM|OMN
P. R. China|CN|CHN
P. R. China, P. R. China|CN|CHN
P.R China|CN|CHN
P.R. China|CN|CHN
P.R. China, P.R. China|CN|CHN
P.R. China., P.R. China|CN|CHN
P.R. of China|CN|CHN
P.R.China|CN|CHN
PR China|CN|CHN
PRC|CN|CHN
Pakistan|PK|PAK
Palestine|PS|PSE
Pennsylvania|US|USA
Pennsylvania;|US|USA
People's Republic of China|CN|CHN
People9s Republic of China|CN|CHN
Peru|PE|PER
Philippines|PH|PHL
Poland|PL|POL
Portugal|PT|PRT
Qatar|QA|QAT
ROC|CN|CHN
Republic of Ireland|IE|GBR
Republic of Kazakhstan|KZ|KAZ
Republic of Korea|KR|KOR
Republic of Panama|PA|PAN
Republic of Singapore|SG|SGP
Republic of The Gambia|GM|GMB
Reunion Island|FR|FRA
Reunion Island, France|FR|FRA
Romania|RO|ROU
Russia|RU|RUS
Saudi Arabia|SA|SAU
Saudi Arabia, Saudi Arabia|SA|SAU
Scotland, UK|GB|GBR
Scotland, United Kingdom|GB|GBR
Sellman BR||
Senegal|SN|SEN
Serbia|RS|SRB
Singapore|SG|SGP
Singapore ¤|SG|SGP
Singapore, Singapore|SG|SGP
Singapore, Singapore, Singapore|SG|SGP
Singapore. Correspondence|SG|SGP
Slovak Republic|SK|SVK
Slovakia|SK|SVK
Slovenia|SI|SVN
South||
South Africa|ZA|ZAF
South China, China|CN|CHN
South Korea|KR|KOR
South Korea. Correspondence|KR|KOR
Spain|ES|ESP
Spain, France|ES|ESP
Spain, UNITED STATES|ES|ESP
Spain, United States of America|ES|ESP
Sri Lanka|LK|LKA
Stratoxon LLC USA, USA, USA|US|USA
Sudan|SD|SDN
Sweden|SE|SWE
Sweden, Germany|SE|SWE
Sweden, Netherlands|SE|SWE
Sweden, Norway|SE|SWE
Switzerland|CH|CHE
Switzerland, Cameroon|CH|CHE
Switzerland., UK|CH|CHE
Taiwan|TW|TWN
Taiwan (R.O.C.|TW|TWN
Taiwan (ROC|TW|TWN
Taiwan R.O.C|TW|TWN
Taiwan ROC|TW|TWN
Taiwan ROC Republic of China|TW|TWN
Taiwan(|TW|TWN
Taiwan, ROC|TW|TWN
Taiwan, Republic of China|TW|TWN
Tanzania|TZ|TZA
Thailand|TH|THA
Thailand (DL|TH|THA
The Gambia|GM|GMB
The Netherlands|NL|NLD
The Netherlands ARTICLE HISTORY|NL|NLD
The Netherlands., The Netherlands|NL|NLD
The P.R. China|CN|CHN
Tunisia|TN|TUN
Turkey|TR|TUR
U.S.A|US|USA
UAE|AE|ARE
UK|GB|GBR
UK ARTICLE HISTORY|GB|GBR
UK ARTICLE HISTORY, UK|GB|GBR
UK, UK|GB|GBR
UK., Germany|GB|GBR
UK;, Germany|GB|GBR
UNITED STATES|US|USA
US, USA|US|USA
USA|US|USA
USA, China|US|USA
USA, USA|US|USA
USA, USA, USA, USA|US|USA
USA.|US|USA
USA., Vietnam|US|USA
USA;, Germany|US|USA
Uganda|UG|UGA
Ukraine|UA|UKR
United Arab Emirates|AE|ARE
United Arab, United Arab Emirates|AE|ARE
United Kingdom|GB|GBR
United Kingdom, United Kingdom|GB|GBR
United Kingdom, United States of America|GB|GBR
United Stated of America}|US|USA
United States|US|USA
United States of America|US|USA
United States of America, Germany|US|USA
United States of America, United States|US|USA
United States of America, United States of America|US|USA
United States, Germany|US|USA
United States, USA|US|USA
United States, United States|US|USA
United States, United States of America|US|USA
United States, United States, Italy, Greece|US|USA
United-Kingdom|GB|GBR
Uruguay|UY|URY
UsA|US|USA
Vietnam|VN|VNM
Virginia, USA|US|USA
australia, australia|AU|AUS
israel|IL|ISR
italy|IT|ITA
the Netherlands|NL|NLD
""", True)

Wrote 5648 bytes.
Out[56]: True